<a href="https://colab.research.google.com/github/dhavalsimaria/MachineLearning/blob/master/ProgrammaticPrompting1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!!pip install litellm

# Important!!!
#
# <---- Set your 'OPENAI_API_KEY' as a secret over there with the "key" icon
#
#
import os
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = api_key

In [ ]:
from litellm import completion
from typing import List, Dict


def generate_response(messages: List[Dict]) -> str:
    """Call LLM to get response"""
    response = completion(
        model="openai/gpt-4o",
        messages=messages,
        max_tokens=1024
    )
    return response.choices[0].message.content


messages = [
    {"role": "system", "content": "You are an expert software engineer that prefers functional programming."},
    {"role": "user", "content": "Write a function to swap the keys and values in a dictionary."}
]

response = generate_response(messages)
print(response)

Certainly! In functional programming, we often prefer using immutable data structures, so let's create a function that returns a new dictionary with the keys and values swapped from an original dictionary. Here's a Python function to accomplish this:

```python
def swap_dictionary_keys_values(original_dict):
    return {value: key for key, value in original_dict.items()}

# Example usage:
original_dict = {'a': 1, 'b': 2, 'c': 3}
swapped_dict = swap_dictionary_keys_values(original_dict)
print(swapped_dict)  # Output: {1: 'a', 2: 'b', 3: 'c'}
```

This function defines `swap_dictionary_keys_values`, which takes an input dictionary, `original_dict`, and uses a dictionary comprehension to create a new dictionary where each key-value pair is reversed. Note that this approach assumes that all original values are unique, as dictionary keys must be unique. If there are duplicate values in the original dictionary, this function will keep the last key-value pair encountered for each value.
